In [1]:
!pip install yfinance
!pip install matplotlib
!pip install scipy
import yfinance as yf, numpy as np, datetime as dt
from scipy.stats import norm
import datetime as dt
import yfinance as yf
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

/Users/himanshuthakur/anaconda3/lib/python3.10/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [9]:
start = dt.datetime(2023,6,1)
end = dt.datetime.now()

df = yf.download('META', start, end)
df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-06-01,265.899994,274.000000,265.890015,272.609985,272.050934,25609500
2023-06-02,272.660004,275.350006,271.119995,272.609985,272.050934,19405300
2023-06-05,270.299988,275.570007,269.559998,271.390015,270.833466,20742900
2023-06-06,270.140015,276.570007,269.690002,271.119995,270.563995,19419000
2023-06-07,271.670013,274.250000,262.799988,263.600006,263.059448,26163600
...,...,...,...,...,...,...
2024-07-24,472.309998,476.299988,460.579987,461.269989,461.269989,17649700
2024-07-25,463.260010,463.549988,442.649994,453.410004,453.410004,18240500
2024-07-26,464.200012,469.769989,459.420013,465.700012,465.700012,14222400


In [10]:
# Define the RSI function
def rsi(series, period=14):
    delta = series.diff(1)
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)
    avg_gain = gain.rolling(window=period, min_periods=1).mean()
    avg_loss = loss.rolling(window=period, min_periods=1).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

In [11]:
# Define the MACD function
def macd(series, slow=26, fast=12, signal=9):
    fast_ema = series.ewm(span=fast, adjust=False).mean()
    slow_ema = series.ewm(span=slow, adjust=False).mean()
    macd_line = fast_ema - slow_ema
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    return macd_line, signal_line

In [12]:
# Define Bollinger Bands
def bollinger_bands(series, window=20, n_std=2):
    rolling_mean = series.rolling(window).mean()
    rolling_std = series.rolling(window).std()
    upper_band = rolling_mean + (rolling_std * n_std)
    lower_band = rolling_mean - (rolling_std * n_std)
    return upper_band, lower_band

In [13]:
# Define the strategy
class MomentumReversionStrategy_2(Strategy):
    rsi_window = 14
    macd_slow = 26
    macd_fast = 12
    macd_sign = 9
    boll_window = 20
    boll_dev = 2

    def init(self):
        close = pd.Series(self.data.Close, name='close')

        # Create custom indicators
        self.rsi = self.I(rsi, close, self.rsi_window)
        macd_line, macd_signal = self.I(macd, close, self.macd_slow, self.macd_fast, self.macd_sign)
        self.macd_line = macd_line
        self.macd_signal = macd_signal
        self.bollinger_high, self.bollinger_low = self.I(bollinger_bands, close, self.boll_window, self.boll_dev)

    def next(self):
        if (self.rsi[-1] < 30) and (self.data.Close[-1] < self.bollinger_low[-1]):
            self.buy()
        elif (self.rsi[-1] > 70) and (self.macd_line[-1] > self.macd_signal[-1]):
            self.sell()

In [14]:
# Run the backtest
bt = Backtest(df, MomentumReversionStrategy_2, cash=10000, commission=.002)

# Optimize the strategy parameters
stats = bt.optimize(
    rsi_window=range(8, 22, 1),
    macd_slow=range(18, 32, 1),
    macd_fast=range(6, 14, 1),
    macd_sign=range(6, 14, 1),
    boll_window=range(10, 31, 1),
    boll_dev=[1, 2],
    maximize='Sharpe Ratio',
    constraint=lambda p: p.macd_fast < p.macd_slow
)

# Print the optimization results
print(stats) 

/Users/himanshuthakur/anaconda3/lib/python3.10/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 526848 configurations.
  output = _optimize_grid()
/Users/himanshuthakur/anaconda3/lib/python3.10/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/1757 [00:00<?, ?it/s]

Start                     2023-06-01 00:00:00
End                       2024-07-30 00:00:00
Duration                    425 days 00:00:00
Exposure Time [%]                   90.410959
Equity Final [$]                  17760.69274
Equity Peak [$]                  20531.271719
Return [%]                          77.606927
Buy & Hold Return [%]               69.909404
Return (Ann.) [%]                   64.167548
Volatility (Ann.) [%]               58.586579
Sharpe Ratio                          1.09526
Sortino Ratio                        3.098848
Calmar Ratio                         3.484915
Max. Drawdown [%]                  -18.412943
Avg. Drawdown [%]                   -3.681243
Max. Drawdown Duration       91 days 00:00:00
Avg. Drawdown Duration       13 days 00:00:00
# Trades                                    8
Win Rate [%]                            100.0
Best Trade [%]                      62.886749
Worst Trade [%]                      6.371639
Avg. Trade [%]                    

In [15]:
bt.plot()

Row(id='1601', ...)

In [16]:
stats._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-4,28,53,312.992755,293.049988,79.771069,0.063716,2023-07-13,2023-08-17,35 days
1,-8,28,54,312.992755,279.029999,271.702051,0.108510,2023-07-13,2023-08-18,36 days
2,-15,28,55,312.992755,283.450012,443.141144,0.094388,2023-07-13,2023-08-21,39 days
3,-4,28,57,312.992755,288.500000,97.971021,0.078253,2023-07-13,2023-08-23,41 days
4,-1,29,57,311.166429,288.500000,22.666429,0.072843,2023-07-14,2023-08-23,40 days
5,-2,33,57,312.873000,288.500000,48.746000,0.077901,2023-07-20,2023-08-23,34 days
6,15,59,291,286.702265,467.000000,2704.466027,0.628867,2023-08-25,2024-07-30,340 days
7,23,57,291,289.077000,467.000000,4092.229000,0.615487,2023-08-23,2024-07-30,342 days
